# Activité : Effectuez un nettoyage et une analyse exploratoire de données texte

In [1]:
# Chargement des bibliothèques utiles pour cette activité :

import pandas as pd
import numpy as np
import nltk
import os
import pickle

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [2]:
path = 'stories'    # dossier contenant les données (fichiers '.story')
story_ids = []
articles = []
highlights = []

# On balaye la liste des fichiers contenant les données :

for filename in os.listdir(path):
    # Récupération du texte brut :
    with open(path + os.path.sep + filename, 'r') as cur_file:
        text_raw = cur_file.read()
    # Création paire article / highlights :
    start_index_highlight = text_raw.find('@highlight')
    article_raw = text_raw[:start_index_highlight]
    highlights_raw = text_raw[start_index_highlight:]
    # Stockage :
    story_ids.append(os.path.splitext(filename)[0])    # nom du fichier sans extension
    articles.append(article_raw)
    highlights.append(highlights_raw)

story_ids = np.array(story_ids, dtype=object)

In [3]:
# On utilise la fonction 'CountVectorizer' de scikit-learn pour effectuer en une seule opération
# à la fois les traitements sur le texte (passage en minuscules, tokenisation, suppression de la
# ponctuation, suppression des stopwords) et le calcul des fréquences des tokens.

# On crée une fonction 'cvect' pour simplifier l'appel à 'CountVectorizer' :

def cvect(corpus, stop_words, max_df):
    cvec = CountVectorizer(
        lowercase=True,        # passage en minuscules
        tokenizer=None,        # on garde le tokenizer par défaut de scikit-learn
        token_pattern=r'\w+',  # tokenisation avec suppression de la ponctuation
        stop_words=stop_words, # suppression des stopwords standards
        max_df=max_df)         # suppression des stopwords spécifiques intra-corpus
    # Calcul des fréquences :
    term_freq = cvec.fit_transform(corpus)
    # Récupération du vocabulaire (tokens) sous forme d'array classée dans l'ordre des indices :
    vocabulary = np.array(zip(*sorted(cvec.vocabulary_.iteritems(), key=lambda (k,v): (v,k)))[0], dtype=object)
    return term_freq, vocabulary

In [4]:
# On définit les stopwords :

# - pour les articles :
sw_articles = set(nltk.corpus.stopwords.words('english'))    # stopwords standards pour l'anglais
max_df_articles = 0.8    # stopwords spécifiques intra-corpus présents dans plus de 80% des documents

# - pour les highlights, l'énoncé ne demande pas la suppression des stopwords :
sw_highlights = set(['highlight'])    # on élimine juste le mot-clé 'highlight'
max_df_highlights = 1.0    # valeur par défaut, pas de stopwords spécifiques intra-corpus

# On applique 'cvect' ('CountVectorizer') aux articles, puis aux highlights :
articles_freq, articles_vocabulary = cvect(articles, sw_articles, max_df_articles)
highlights_freq, highlights_vocabulary = cvect(highlights, sw_highlights, max_df_highlights)

In [5]:
# Pour le calcul des tf-idf, on utilise la fonction 'TfidfTransformer' de scikit-learn, qui équivaut
# à 'TfidfVectorizer' lorsqu'elle est appliquée à la suite de 'CountVectorizer' :

articles_tfidf = TfidfTransformer().fit_transform(articles_freq)
highlights_tfidf = TfidfTransformer().fit_transform(highlights_freq)

In [6]:
# Création d'un dictionnaire regroupant le nouveau jeu de données d'entraînement :

dataset = {
    # articles
    'articles_tfidf' : articles_tfidf,               # matrice creuse tf-idf (dim = n_stories x n_tokens_articles)
    'articles_vocabulary' : articles_vocabulary,     # liste (dim = n_tokens_articles) des features (tokens)
    # highlights
    'highlights_tfidf' : highlights_tfidf,           # matrice creuse tf-idf (dim = n_stories x n_tokens_highlights)
    'highlights_vocabulary' : highlights_vocabulary, # liste (dim = n_tokens_highlights) des features (tokens)
    # références des stories (pour traçabilité)
    'story_ids' : story_ids                          # liste (dim = n_stories) des noms de fichiers sans extension
}

In [7]:
# Enregistrement du nouveau jeu de données d'entraînement dans un fichier pour usage ultérieur :

with open('dataset.pkl', 'wb') as fichier:
    mon_pickler = pickle.Pickler(fichier)
    mon_pickler.dump(dataset)